# COVID-19 可视化分析及模型预测

### **Time: 2021/01/09**

<br>

# Table of Contents
**[1.全球疫情速览：是谁为人民服务，是谁一地鸡毛](#id_overview)**<br/>
**[2.COVID-19相关文献的文本研究：目光所言何处](#id_literature)**<br/>
**[3.未来确诊人数发展预测：黎明前的黑暗](#id_prediction)**<br/>
**[4.医药统计相关职位分析：数据科学家的新赛道](#id_medicine)**<br/>

In [ ]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_white"
from plotly.subplots import make_subplots

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

# --- wordcloud ---
import os
from os import path
import numpy as np
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from PIL import Image
from matplotlib import pyplot as plt
from imageio import imread
import random

# --- pyecharts ---
import pandas as pd
from pyecharts.charts import Pie
from pyecharts import options as opts

In [ ]:
#pip install pyecharts
#pip install pillow wordcloud imageio jieba snownlp itchat -i https://pypi.tuna.tsinghua.edu.cn/simple
#pip install imageio

<a id="id_overview"></a>
# 1. 全球疫情时空数据可视化

## 1.1 数据预处理

In [ ]:
# --- Globel ---
from datetime import datetime
def _convert_date_str(df):
    try:
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
    except:
        print('_convert_date_str failed with %y, try %Y')
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
confirmed_global_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
_convert_date_str(confirmed_global_df)

deaths_global_df = pd.read_csv('time_series_covid19_deaths_global.csv')
_convert_date_str(deaths_global_df)

recovered_global_df = pd.read_csv('time_series_covid19_recovered_global.csv')
_convert_date_str(recovered_global_df)

# Filter out problematic data points (The West Bank and Gaza had a negative value, cruise ships were associated with Canada, etc.)
removed_states = "Recovered|Grand Princess|Diamond Princess"
removed_countries = "US|The West Bank and Gaza"

confirmed_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
deaths_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
recovered_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
deaths_global_df    = deaths_global_df[~deaths_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
recovered_global_df = recovered_global_df[~recovered_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
deaths_global_df    = deaths_global_df[~deaths_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
recovered_global_df = recovered_global_df[~recovered_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]

confirmed_global_melt_df = confirmed_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='ConfirmedCases')
deaths_global_melt_df = deaths_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Deaths')
recovered_global_melt_df = deaths_global_df.melt(
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Recovered')

In [ ]:
train = confirmed_global_melt_df.merge(deaths_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date'])
train = train.merge(recovered_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date'])

In [ ]:
train

In [ ]:
# --- US ---
confirmed_us_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths_us_df = pd.read_csv('time_series_covid19_deaths_US.csv')

confirmed_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'], inplace=True, axis=1)
deaths_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'], inplace=True, axis=1)

confirmed_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True)
deaths_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True)

_convert_date_str(confirmed_us_df)
_convert_date_str(deaths_us_df)

# clean
confirmed_us_df = confirmed_us_df[~confirmed_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]
deaths_us_df = deaths_us_df[~deaths_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]

# --- Aggregate by province state ---
#confirmed_us_df.groupby(['Country_Region', 'Province_State'])
confirmed_us_df = confirmed_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()
deaths_us_df = deaths_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()

# remove lat, long.
confirmed_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)
deaths_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)

confirmed_us_melt_df = confirmed_us_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_us_df.columns[2:], var_name='Date', value_name='ConfirmedCases')
deaths_us_melt_df = deaths_us_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=deaths_us_df.columns[2:], var_name='Date', value_name='Deaths')

train_us = confirmed_us_melt_df.merge(deaths_us_melt_df, on=['Country_Region', 'Province_State', 'Date'])

In [ ]:
train_us

In [ ]:
# Globel与US合并
train = pd.concat([train, train_us], axis=0, sort=False)

train_us.rename({'Country_Region': 'country', 'Province_State': 'province', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities'}, axis=1, inplace=True)
train_us['country_province'] = train_us['country'].fillna('') + '/' + train_us['province'].fillna('')

In [ ]:
train.rename({'Country_Region': 'country', 'Province_State': 'province', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities', 'Recovered': 'recovered'}, axis=1, inplace=True)
train['country_province'] = train['country'].fillna('') + '/' + train['province'].fillna('')

In [ ]:
train

## 1.2 全球变化趋势/各个国家变化趋势

### 1.2.1 全球变化趋势

In [ ]:
# 疫情首先出现在中国，并在很短的时间内便席卷了亚洲。进入二月份以后，尽管打着”反中抗中“旗号赚取选票的
# 政客持续反对，但当欧美国家的核酸检测在CDC的强烈推动下较大面积铺开后，这些国家（以美国为代表）
# 的确诊人数数据急剧攀升，并最终演化为今天的疫情重灾区。

In [ ]:
country_df = train.groupby(['date', 'country'])[['confirmed', 'fatalities']].sum().reset_index()
countries = country_df['country'].unique()
target_date = country_df['date'].max()

In [ ]:
country_df['date'] = country_df['date'].apply(str)
country_df['confirmed'] = country_df['confirmed']
country_df['fatalities'] = country_df['fatalities']

fig = px.scatter_geo(country_df, locations="country", locationmode='country names', 
                     color="confirmed", size='confirmed', hover_name="country", 
                     hover_data=['confirmed', 'fatalities'],
                     range_color= [0, country_df['confirmed'].max()], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Confirmed cases spread Over Time', color_continuous_scale="portland")
# fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
fig = px.scatter_geo(country_df, locations="country", locationmode='country names', 
                     color="fatalities", size='fatalities', hover_name="country", 
                     hover_data=['confirmed', 'fatalities'],
                     range_color= [0, country_df['fatalities'].max()], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Fatalities growth Over Time', color_continuous_scale="portland")
fig.show()

In [ ]:
country_df['prev_confirmed'] = country_df.groupby('country')['confirmed'].shift(1)
country_df['new_case'] = country_df['confirmed'] - country_df['prev_confirmed']
country_df['new_case'].fillna(0, inplace=True)

country_df.loc[country_df['new_case'] < 0, 'new_case'] = 0.
fig = px.scatter_geo(country_df, locations="country", locationmode='country names', 
                     color="new_case", size='new_case', hover_name="country", 
                     hover_data=['confirmed', 'fatalities'],
                     range_color= [0, country_df['new_case'].max()], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: Daily NEW cases over Time', color_continuous_scale="portland")
fig.show()

In [ ]:
ww_df = train.groupby('date')[['confirmed', 'fatalities']].sum().reset_index()
ww_df['new_case'] = ww_df['confirmed'] - ww_df['confirmed'].shift(1)
ww_df['growth_factor'] = ww_df['new_case'] / ww_df['new_case'].shift(1)

ww_df['mortality'] = round(ww_df['fatalities'] / ww_df['confirmed'], 4)

fig = px.line(ww_df, x="date", y="mortality", 
              title="Worldwide Mortality Rate Over Time")
fig.show()

In [ ]:
## 二月份的死亡率快速下降得益于以中国大陆为首的亚洲疫情重灾区国家相对严格的封锁政策、减少人员流动
## 然而，三月份开始，当亚洲以外的国家大量进行检测时，成倍的死亡被追踪到为新冠病毒。

In [ ]:
fig = px.line(ww_df, x="date", y="growth_factor", 
              title="Worldwide Growth Factor Over Time")
fig.add_trace(go.Scatter(x=[ww_df['date'].min(), ww_df['date'].max()], y=[1., 1.], name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)'))))
fig.update_yaxes(range=[0., 5.])
fig.show()

In [ ]:
# NOTE：
#Growth factor怎么得到的？
#Boris Johnson: 3.16宣布自然免疫策略；4月8日确诊；Trump：10月12日确诊

In [ ]:
#Growth factor由每日增量与前一日存量做比率得到。五月份以前的剧烈波动源于之前提到过的严格的封城与欧美国家大量检测导致的增长因子大幅波动。
#而另一方面，五月至十一月的增长率维持在1左右。相应时间段的夏季高温与普遍封城是控制疫情成功的主要原因。

### 1.2.2 各国家变化趋势

In [ ]:
country_df = train.groupby(['date', 'country'])[['confirmed', 'fatalities']].sum().reset_index()
country_df.tail()

countries = country_df['country'].unique()
#print(f'{len(countries)} countries are in dataset:\n{countries}')

target_date = country_df['date'].max()

print('Date: ', target_date)
for i in [1, 100, 10000, 100000, 1000000, 10000000]:
    n_countries = len(country_df.query('(date == @target_date) & confirmed > @i'))
    print(f'{n_countries} countries have more than {i} confirmed cases')

In [ ]:
all_country_df = country_df.query('date == @target_date')
all_country_df['confirmed_log1p'] = np.log10(all_country_df['confirmed'] + 1)
all_country_df['fatalities_log1p'] = np.log10(all_country_df['fatalities'] + 1)
all_country_df['mortality_rate'] = all_country_df['fatalities'] / all_country_df['confirmed']

In [ ]:
fig = px.choropleth(all_country_df, locations="country", 
                    locationmode='country names', color="confirmed_log1p", 
                    hover_name="country", hover_data=["confirmed", 'fatalities', 'mortality_rate'],
                    range_color=[all_country_df['confirmed_log1p'].min(), all_country_df['confirmed_log1p'].max()], 
                    color_continuous_scale="peach", 
                    title='Countries with Confirmed Cases')

# I'd like to update colorbar to show raw values, but this does not work somehow...
# Please let me know if you know how to do this!!
trace1 = list(fig.select_traces())[0]
trace1.colorbar = go.choropleth.ColorBar(
    tickvals=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    ticktext=['1', '10', '100', '1000','10000', '100000', '1000000', '10000000', '100000000'])
fig.show()

In [ ]:
fig = px.choropleth(all_country_df, locations="country", 
                    locationmode='country names', color="mortality_rate", 
                    hover_name="country", range_color=[0, 0.10], 
                    color_continuous_scale="peach", 
                    title='Countries with mortality rate')
fig.show()

In [ ]:
country_latest = country_df.query('date == @target_date')

fig = px.treemap(country_latest, path=["country"], values='confirmed', height=700,
                 title='Confirmed', color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
# 这两张树状图清晰地展现了以美国英国为代表的西方发达国家、以印度巴西为代表的医疗设施落后
# 的发展中国家在确诊人数和死亡率上不容乐观的表现。

In [ ]:
fig = px.treemap(country_latest, path=["country"], values='fatalities', height=700,
                 title='Fatalities', color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
ax = sns.distplot(np.log10(country_df.query('date == "2021-01-07"')['confirmed'] + 1))
ax.set_title("confirmed cases histogram on 2021/1/7")
ax.set_xlim([0, 8])
ax.set_xticks(np.arange(9))
_ = ax.set_xticklabels(['0', '10', '100', '1k', '10k', '100k', '1m', '10m'])

In [ ]:
top_country_df = country_df.query('(date == @target_date) & (confirmed > 100)')
top_country_df['mortality_rate'] = round(top_country_df['fatalities'] / top_country_df['confirmed'], 4)
top_country_df = top_country_df.sort_values('mortality_rate', ascending=False)

fig = px.bar(top_country_df[:30].iloc[::-1],
             x='mortality_rate', y='country',
             title=f'Mortality rate HIGH: top 30 countries on {target_date}', text='mortality_rate', height=800, orientation='h')
fig.show()

In [ ]:
n_countries = 20
n_start_death = 10
fatality_top_countires = top_country_df.sort_values('fatalities', ascending=False).iloc[:n_countries]['country'].values
country_df['date'] = pd.to_datetime(country_df['date'])


df_list = []
for country in fatality_top_countires:
    this_country_df = country_df.query('country == @country')
    start_date = this_country_df.query('fatalities > @n_start_death')['date'].min()
    this_country_df = this_country_df.query('date >= @start_date')
    this_country_df['date_since'] = this_country_df['date'] - start_date
    this_country_df['fatalities_log1p'] = np.log10(this_country_df['fatalities'] + 1)
    this_country_df['fatalities_log1p'] -= this_country_df['fatalities_log1p'].values[0]
    df_list.append(this_country_df)

tmpdf = pd.concat(df_list)
tmpdf['date_since_days'] = tmpdf['date_since'] / pd.Timedelta('1 days')

In [ ]:
top30_countries = top_country_df.sort_values('confirmed', ascending=False).iloc[:30]['country'].unique()
country_df['prev_confirmed'] = country_df.groupby('country')['confirmed'].shift(1)
country_df['new_case'] = country_df['confirmed'] - country_df['prev_confirmed']
country_df['new_case'].fillna(0, inplace=True)
top30_country_df = country_df[country_df['country'].isin(top30_countries)]

fig = px.line(top30_country_df,
              x='date', y='new_case', color='country',
              title=f'DAILY NEW Confirmed cases by country')
fig.show()

In [ ]:
country_df['avg_new_case'] = country_df.groupby('country')['new_case'].rolling(7).mean().reset_index(0, drop=True)
country_df['prev_new_case'] = country_df.groupby('country')['avg_new_case'].shift(1)
country_df['growth_factor'] = country_df['avg_new_case'] / country_df['prev_new_case']

country_df['growth_factor'].fillna(0, inplace=True)
top30_country_df = country_df[country_df['country'].isin(top30_countries)]

fig = px.line(top30_country_df,
              x='date', y='growth_factor', color='country',
              title=f'Growth factor by country')
fig.add_trace(go.Scatter(x=[ww_df['date'].min(), ww_df['date'].max()], y=[1., 1.], name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)'))))
fig.update_yaxes(range=[0., 5.])
fig.show()

## 1.3 重点地区与重要时段

### 1.3.1  美国

In [ ]:
for country in countries:
    province = train.query('country == @country')['province'].unique()
    if len(province) > 1:       
        print(f'Country {country} has {len(province)} provinces: {province}')

In [ ]:
usa_state_code_df = pd.read_csv('usa_states2.csv')

In [ ]:
train_us

In [ ]:
# Prepare data frame only for US. 

#train_us = train.query('country == "US"')
train_us['mortality_rate'] = train_us['fatalities'] / train_us['confirmed']

# Convert province column to its 2-char code name,
state_name_to_code = dict(zip(usa_state_code_df['state_name'], usa_state_code_df['state_code']))
train_us['province_code'] = train_us['province'].map(state_name_to_code)

# Only show latest days.
train_us_latest = train_us.query('date == @target_date')

In [ ]:
fig = px.choropleth(train_us_latest, locations='province_code', locationmode="USA-states",
                    color='confirmed', scope="usa", hover_data=['province', 'fatalities', 'mortality_rate'],
                    title=f'Confirmed cases in US on {target_date}')
fig.show()

In [ ]:
train_us_latest.sort_values('confirmed', ascending=False)

In [ ]:
fig = px.choropleth(train_us_latest, locations='province_code', locationmode="USA-states",
                    color='mortality_rate', scope="usa", hover_data=['province', 'fatalities', 'mortality_rate'],
                    title=f'Mortality rate in US on {target_date}')
fig.show()

In [ ]:
train_us_march = train_us.query('date > "2020-03-01"')
fig = px.line(train_us_march,
              x='date', y='confirmed', color='province',
              title=f'Confirmed cases by state in US, as of {target_date}')
fig.show()

In [ ]:
train_us_march['prev_confirmed'] = train_us_march.groupby('province')['confirmed'].shift(1)
train_us_march['new_case'] = train_us_march['confirmed'] - train_us_march['prev_confirmed']
train_us_march['new_case'].fillna(0, inplace=True)

fig = px.line(train_us_march,
              x='date', y='new_case', color='province',
              title=f'DAILY NEW Confirmed cases by states in US')
fig.show()

In [ ]:
train_us_march['avg_new_case'] = train_us_march.groupby('province')['new_case'].rolling(7).mean().reset_index(0, drop=True)
train_us_march['prev_new_case'] = train_us_march.groupby('province')['avg_new_case'].shift(1)
train_us_march['growth_factor'] = train_us_march['avg_new_case'] / train_us_march['prev_new_case']
train_us_march['growth_factor'].fillna(0, inplace=True)
fig = px.line(train_us_march,
              x='date', y='growth_factor', color='province',
              title=f'Growth factor by state in US')
fig.add_trace(go.Scatter(x=[train_us_march['date'].min(), train_us_march['date'].max()], y=[1., 1.],
                         name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)'))))
fig.update_yaxes(range=[0., 5.])
fig.show()

### 1.3.2  欧洲

In [ ]:
# Ref: https://www.kaggle.com/abhinand05/covid-19-digging-a-bit-deeper
europe_country_list =list([
    'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland',
    'Italy', 'Latvia','Luxembourg','Lithuania','Malta','Norway','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia',
    'Spain', 'Sweden', 'United Kingdom', 'Iceland', 'Russia', 'Switzerland', 'Serbia', 'Ukraine', 'Belarus',
    'Albania', 'Bosnia and Herzegovina', 'Kosovo', 'Moldova', 'Montenegro', 'North Macedonia'])

country_df['date'] = pd.to_datetime(country_df['date'])
train_europe = country_df[country_df['country'].isin(europe_country_list)]
#train_europe['date_str'] = pd.to_datetime(train_europe['date'])
train_europe_latest = train_europe.query('date == @target_date')

In [ ]:
fig = px.choropleth(train_europe_latest, locations="country", 
                    locationmode='country names', color="confirmed", 
                    hover_name="country", range_color=[1, train_europe_latest['confirmed'].max()], 
                    color_continuous_scale='portland', 
                    title=f'European Countries with Confirmed Cases as of {target_date}', scope='europe', height=800)
fig.show()

In [ ]:
train_europe_march = train_europe.query('date >= "2020-03-01"')
fig = px.line(train_europe_march,
              x='date', y='confirmed', color='country',
              title=f'Confirmed cases by country in Europe, as of {target_date}')
fig.show()

In [ ]:
fig = px.line(train_europe_march,
              x='date', y='fatalities', color='country',
              title=f'Fatalities by country in Europe, as of {target_date}')
fig.show()

In [ ]:
train_europe_march['prev_confirmed'] = train_europe_march.groupby('country')['confirmed'].shift(1)
train_europe_march['new_case'] = train_europe_march['confirmed'] - train_europe_march['prev_confirmed']
fig = px.line(train_europe_march,
              x='date', y='new_case', color='country',
              title=f'DAILY NEW Confirmed cases by country in Europe')
fig.show()

In [ ]:
train_europe_march['avg_new_case'] = train_europe_march.groupby('country')['new_case'].rolling(7).mean().reset_index(0, drop=True)
train_europe_march['prev_new_case'] = train_europe_march.groupby('country')['avg_new_case'].shift(1)
train_europe_march['growth_factor'] = train_europe_march['avg_new_case'] / train_europe_march['prev_new_case']
train_europe_march['growth_factor'].fillna(0, inplace=True)
fig = px.line(train_europe_march,
              x='date', y='growth_factor', color='country',
              title=f'Growth factor by country in Europe')
fig.add_trace(go.Scatter(x=[train_europe_march['date'].min(), train_europe_march['date'].max()], y=[1., 1.],
                         name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)'))))
fig.update_yaxes(range=[0., 5.])
fig.show()

### 1.3.3  亚洲

In [ ]:
country_latest = country_df.query('date == @target_date')

fig = px.choropleth(country_latest, locations="country", 
                    locationmode='country names', color="confirmed", 
                    hover_name="country", range_color=[1, 300000], 
                    color_continuous_scale='portland', 
                    title=f'Asian Countries with Confirmed Cases as of {target_date}', scope='asia', height=800)
fig.show()

In [ ]:
top_asian_country_df = country_df[country_df['country'].isin([
    'China', 'Indonesia', 'Iran', 'Japan', 'Korea, South', 'Malaysia', 'Philippines',
    'India', 'Bangladesh', 'Pakistan', 'Saudi Arabia', 'Turkey'
])]

fig = px.line(top_asian_country_df,
              x='date', y='new_case', color='country',
              title=f'DAILY NEW Confirmed cases Asia')
fig.show()

In [ ]:
top_asian_country_df['avg_new_case'] = top_asian_country_df.groupby('country')['new_case'].rolling(7).mean().reset_index(0, drop=True)
top_asian_country_df['prev_new_case'] = top_asian_country_df.groupby('country')['avg_new_case'].shift(1)
top_asian_country_df['growth_factor'] = top_asian_country_df['avg_new_case'] / top_asian_country_df['prev_new_case']
top_asian_country_df['growth_factor'].fillna(0, inplace=True)
fig = px.line(top_asian_country_df,
              x='date', y='growth_factor', color='country',
              title=f'Growth factor by country in Asia')
fig.add_trace(go.Scatter(x=[top_asian_country_df['date'].min(), top_asian_country_df['date'].max()], y=[1., 1.],
                         name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)'))))
fig.update_yaxes(range=[0., 5.])
fig.show()

## 1.4 正在恢复的国家

In [ ]:
china_df = train.query('country == "China"')
china_df['prev_confirmed'] = china_df.groupby('province')['confirmed'].shift(1)
china_df['new_case'] = china_df['confirmed'] - china_df['prev_confirmed']
china_df.loc[china_df['new_case'] < 0, 'new_case'] = 0.

In [ ]:
fig = px.line(china_df,
              x='date', y='new_case', color='province',
              title=f'DAILY NEW Confirmed cases in China by province')
fig.show()

<br>

<a id="id_literature"></a>
# 2. COVID-19相关文献的文本研究

**章节说明：**在文献分析这一部分，分析主要围绕文献的特点、词频等展开，考察了文献的字数分布、文献摘要出现的高频词、文献中国家被提及的次数、文献中其它常见疾病被提及的次数，和文献引用的参考文献。通过这一部分的分析，可以对于新冠肺炎的相关文献有着一个初步的了解。  
**数据来源：**文献分析部分的数据来源于Kaggle

## 2.1 数据预处理

json文件的形式如下，基本是字典嵌套字典的形式，涉及文献的id、标题、作者、摘要、分段的正文和参考文献等。

In [ ]:
# JSON schema of full text documents


{
    "paper_id": <str>,                      # 40-character sha1 of the PDF
    "metadata": {
        "title": <str>,
        "authors": [                        # list of author dicts, in order
            {
                "first": <str>,
                "middle": <list of str>,
                "last": <str>,
                "suffix": <str>,
                "affiliation": <dict>,
                "email": <str>
            },
            ...
        ],
        "abstract": [                       # list of paragraphs in the abstract
            {
                "text": <str>,
                "cite_spans": [             # list of character indices of inline citations
                                            # e.g. citation "[7]" occurs at positions 151-154 in "text"
                                            #      linked to bibliography entry BIBREF3
                    {
                        "start": 151,
                        "end": 154,
                        "text": "[7]",
                        "ref_id": "BIBREF3"
                    },
                    ...
                ],
                "ref_spans": <list of dicts similar to cite_spans>,     # e.g. inline reference to "Table 1"
                "section": "Abstract"
            },
            ...
        ],
        "body_text": [                      # list of paragraphs in full body
                                            # paragraph dicts look the same as above
            {
                "text": <str>,
                "cite_spans": [],
                "ref_spans": [],
                "eq_spans": [],
                "section": "Introduction"
            },
            ...
            {
                ...,
                "section": "Conclusion"
            }
        ],
        "bib_entries": {
            "BIBREF0": {
                "ref_id": <str>,
                "title": <str>,
                "authors": <list of dict>       # same structure as earlier,

**预处理：**考虑只保留文献id、文献标题、文献摘要、文献全文和参考文献几部分。其中，若文献摘要缺失，采用赋值为空字符串处理；文献全文为文献正文分段对拼接；参考文献为参考文献标题的集合。考虑到巨大的数据量对于后续绘图速度的影响，在此只取10000份样本（即两个文件夹各读取5000份）。

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
names=["pmc_json","pdf_json"]
docs=[]
for d in names:
    print(d)
    counts = 0
    for file in tqdm(os.listdir(f"C:/Users/ASUS/Downloads/新建文件夹/archive/document_parses/{d}")):
        file_path = f"C:/Users/ASUS/Downloads/新建文件夹/archive/document_parses/{d}/{file}"
        j = json.load(open(file_path,"rb"))
        paper_id = j['paper_id']
        paper_id = paper_id[-7:]
        title = j['metadata']['title']

        try:
            abstract = j['abstract'][0]['text']  
        except:
            abstract = "" #sometimes there are no abstracts
            
        full_text = ""
        bib_entries = []
        for text in j['body_text']:
            full_text += text['text']
            for csp in text['cite_spans']:
                try:
                    title = j['bib_entries'][csp['ref_id']]['title']
                    bib_entries.append(title)
                except:
                    pass
                
        docs.append([paper_id, title, abstract, full_text, bib_entries])   
        
        #too many data lead to tremendous running time, thus only 10000 articles of each files are taken into accout
        counts = counts + 1
        if(counts > 5000): 
           break
        
df=pd.DataFrame(docs,columns=['paper_id','title','abstract','full_text','bib_entries']) 

经预处理，数据如下表所示：

In [ ]:
df.head()

## 2.2 文献字数分布图

在这一部分中，考虑分析文献的摘要和全文的字数分布，从而可以对文献的长度有一个基本的了解。

In [ ]:
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

fig = make_subplots(rows=2, cols=1, shared_yaxes=False)
df["abstract_num"] = df["abstract"].apply(new_len)
nums1 = df.query("abstract_num != 0 and abstract_num < 500")["abstract_num"]
fig = ff.create_distplot(hist_data=[nums1],
                         group_labels=["All abstracts"],
                         colors=["coral"])
fig.update_layout(title_text="Abstract words", xaxis_title="Abstract words",  yaxis_range=[0,0.008],
                  template="simple_white", showlegend=False)
fig.show()


df["full_num"] = df["full_text"].apply(new_len)
nums2 = df.query("full_num != 0 and full_num < 15000")["full_num"]
fig = ff.create_distplot(hist_data=[nums2],
                         group_labels=["All full texts"],
                         colors=["coral"])
fig.update_layout(title_text="Article words", xaxis_title="Article words", yaxis_range=[0,0.0003],
                  template="simple_white", showlegend=False)
fig.show()

**图表分析：**由上图所示，文章摘要的长度最多集中于155个词附近，而有点出人意料的是不少文献的摘要长度集中于40个词附近。总体来看，文献字数大致在300字以内，与我们认知中的文献字数大致相符，体现了摘要言简意赅的特点。
文章的长度以900-3000字的分布最为集中，超过10000字的文献明显较少，从中可以看出文献大致以中短篇幅为主。

## 2.2  摘要词云图

词云图可以用于展示文章中出现高频词，直观地反映文章涉及的大致内容。在此选取摘要做词云图是考虑到运算的速度，如果运用到全文也是大同小异的。

In [ ]:
#Word cloud for the abstracts
from wordcloud import WordCloud
ab=' '.join(list(df['abstract']))
wordcloud=WordCloud(background_color="white",width=1000,height=500,max_words=100).generate(ab)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

**图表分析：**从上图可以看出，出现频率最高的是COVID和patient，除此之外SARS也是出现频率很高的词，可以想像这是因为SARS和新冠一样都是冠状病毒，有着一定的相似性。另外，有诸如method、model、result、treatment之类的高频词，这反映了不少文献涉及了模型或者方法，探讨了新冠的疗法。

## 2.3 国家出现频数分析

**说明：**这一部分考虑分析文献正文中各国家被提及的次数，该分析可以看出在疫情中哪些国家较为令人瞩目，也可以从某些方面反映各国在疫情方面是否具有代表性。

**分析过程：**
首先对数据进行处理，通过pycountry模块可以获得国家的名称、代码等信息，为后续分析提供帮助。  
然后定义word_count和get_target_dict函数用于从文本中寻找特定的国家名，返回字典形式，为之后的计数处理提供支持。  
最后，计算各个国家在文献正文中出现的总次数，得到一个列表如下：

In [ ]:
import pycountry
def word_count(targets, df, text, key):
    df_targets = df[df[text].apply(lambda sentence: any(word in sentence for word in targets))] 
    original = {}
    word_distribution = {}
    for index, row in df_targets.iterrows():
        original, word_distribution = get_target_dict(targets, row[text], row[key], original, word_distribution)

    return original, word_distribution

import copy
def get_target_dict(targets, text, paper_id, original, word_distribution):
    word_count_new = {}
    word_count = {}
    for sentence in text.split('.'):
        for word in targets:
            if word in sentence:
                if word in word_count: 
                    word_count[word] += sentence.count(word)
                else:
                    word_count[word] = sentence.count(word)
        word_count_new = copy.deepcopy(word_count)
        if bool(word_count_new):
            for word in targets:
                if word not in word_count_new:
                    word_count_new[word] = 0
    word_distribution[paper_id] = word_count_new
    original[paper_id] = word_count 
    return original, word_distribution

country_name=[]
for country in pycountry.countries:
    country_name.append(country.name)
targets=country_name
countries_mentioned,b = word_count(targets,df,'full_text','paper_id')
total_count = {}
for value in countries_mentioned.values():
    total_count = {key: total_count.get(key, 0) + value.get(key, 0)
          for key in set(total_count) | set(value)}
for country,counts in total_count.items():
    total_count[country]=[counts]
print("{:<35} {:<7}".format('Country','Total no. of times mentioned'))
for k, v in total_count.items():
     print("{:<35} {:<7}".format(k, v[0]))

借助于上述数据，进行绘图。

首先绘制地区分布图。该图是对世界地图的填色，具有一定的美观性。同时，地图清晰明了地将地理特点和词频高低结合起来，很好地体现了数据本身所具有的空间的概念。

In [ ]:
import plotly.express as px
np.random.seed(12)
gapminder = px.data.gapminder().query("year==2007")[['country','continent','iso_alpha']]

d = total_count

data_country = pd.DataFrame(d).T.reset_index()
data_country.columns=['country', 'count']

df_merge=pd.merge(gapminder, data_country, on='country', how='left').dropna()

fig = px.choropleth(df_merge, locations="iso_alpha",
                    color="count", 
                    hover_name="country",
                    color_continuous_scale=px.colors.sequential.Emrld,
                    projection='natural earth',
                    title='Frequency of occurence of country')

fig.show()

**图表分析：**  
从上图中，可以看到中国特别显眼，明显文献中中国被提及的次数原高于其它国家，这既有可能是因为疫情最初大面积爆发是在中国，也有可能因为中国在抗疫期间卓越的成效引人瞩目。  
总体来看，亚洲、美洲、欧洲的颜色较深，这意味着这些大洲的国家往往被提及了更多次。这既有可能是因为大部分受瞩目的发达国家集中在这些洲，也很有可能是因为这些洲的不少国家疫情呈现井喷趋势。  
其次，绘制旭日图。旭日图可以看作是升级版的饼图或是环形图，相比此二者，旭日图可以反映层级关系、从属关系。距离圆心较近者具有较高层级，距离圆心较远者具有较低层级。  
在此，同时考虑国家与大洲。显然大洲是较高层级，在内侧。  
下图可以非常直观地反映出被提及次数较多的国家的情况，可以反映出被提及次数较多的大洲的情况。  
由于是交互图，点击某一个大洲名，还可以查看该大洲各国家被提及次数的环形图，非常直观清晰。

In [ ]:
fig = px.sunburst(df_merge, path=['continent', 'country'], values='count',
                  color='count', hover_data=['iso_alpha'],
                  color_continuous_scale='YlOrBr',
                  title='Frequency of occurence of country/continent')
                  
fig.show()

## 2.4 其它疾病频数图

**说明：**  
这一部分考虑新冠肺炎和其它一些常见疾病之间的关系，这里的关系是多种多样的，既可以是新冠会导致的疾病，也可以是某些可能更易导致新冠的疾病。而在文献研究中，不对上述关系作区分，所以只需考虑其出现的频数。  
在此，考虑的常见疾病包括：哮喘、癌症、慢性病、糖尿病、痴呆、心脏病、高血压、呼吸道疾病。

In [ ]:
diseases = ['asthma','cancer','chronic','diabetes','dementia','heart','hypertension','respiratory disease']
countdis=[]
for i in range(len(diseases)):
    countdis.append(''.join(list(df['full_text'])).lower().count(diseases[i].lower()))
diseases = ['Asthma','Cancer','Chronic Disease','Diabetes','Dementia','Heart Disease','Hypertension','Respiratory Disease']
data_disease=pd.DataFrame({'diseases':diseases, 'number':countdis}).sort_values('number',ascending=False)
data_disease.plot(x = 'diseases', 
    y = 'number', 
    kind='bar', 
    legend = False,
    width=0.8)
plt.xlabel("Diseases")
plt.ylabel("Number of occurence")
plt.title("Number of occurence of diseases")
plt.style.use('seaborn-white')
plt.xticks(rotation=45)
plt.show()

**图表分析：**
上图反映在新冠文献中被提及最多的是癌症，其次是慢性病，再次是心脏病。

## 2.5 热门参考文献

**说明：**这一部分考察新冠参考文献所引用的文献，希望找到被引次数最多的一系列文献。  
在此，考虑被引次数最多的10篇文献。  
因为图片显示的原因，文献的名称有截断，但这基本不妨碍我们对于文献内容的大致了解。

In [ ]:
from collections import Counter
bibs=[]
for item in df['bib_entries']:
    for bib in item:
        bibs.append(bib)
a = dict(Counter(bibs))
del a['']
df_a=pd.DataFrame.from_dict(a, orient='index',columns=['no. of times cited'])
df_a['no. of times cited'] = df_a['no. of times cited'].astype(str).astype(int)
sorted_df_a=df_a.sort_values(by='no. of times cited', ascending=False)
new_df = sorted_df_a.loc[sorted_df_a['no. of times cited'] >= 10] 
new_df['title'] = new_df.index.str.slice(0,50)

import seaborn as sns
plt.style.use('seaborn-white')
sns.set()  
new_df[0:10].set_index('no. of times cited').reset_index().sort_index(ascending=False).plot(
    x = 'title', 
    y = 'no. of times cited', 
    kind='barh', 
    legend = False,
    width=0.8
)
plt.style.use('seaborn-white')
plt.xlabel("Number of times cited")
plt.ylabel("Paper")
plt.title("Number of citations of top 10 papers")
plt.gca().yaxis.grid(linestyle='')
plt.gca().xaxis.grid(linestyle=':')
plt.show()

**图表分析：**从上图可以看出，被引次数多的文献主要是关于新冠的临床症状。这反映了新冠作为一种新的传染病，学界对于其的了解处在一个相对比较初级的阶段，所以其症状会被关心。

**章节小结：**综上，这一部分大致对目前的与新冠相关的文献进行了一个初步的分析，可以从中初步了解文献的特点和文献研究的热点。

<br>

<a id="id_prediction"></a>
# 3. COVID-19各国确诊人数的预测

**章节说明：**在了解到当今疫情的现状以及知道了如今研究新冠肺炎的文献都有哪些特点之后，我们或许可以做的更多，去分析未来各个国家的确诊人数的发展方向，这些国家的确诊病例是否会在长期内继续增长，增长的速率又是如何，又会在哪一时期趋于稳定，接下来就通过这个预测模型来进行说明。

## 3.1 长期预测：Sigmoid Fitting

基本的思路是用sigmoid函数来拟合每个国家的确诊病例数（sigmoid函数呈现S型，平滑、易于求导，常用于激活函数）。从病毒的传播模型来看，我们采用了sigmoid曲线去拟合数据是比较自然的，它也是很多研究者所采用的的方法之一。

In [ ]:
top30_countries = top_country_df.sort_values('confirmed', ascending=False).iloc[:30]['country'].unique()

In [ ]:
def sigmoid(t, M, beta, alpha, offset=0):
    alpha += offset
    return M / (1 + np.exp(-beta * (t - alpha)))

def error(x, y, params):
    M, beta, alpha = params
    y_pred = sigmoid(x, M, beta, alpha)

    # apply weight, latest number is more important than past.
    weight = np.arange(len(y_pred)) ** 2
    loss_mse = np.mean((y_pred - y) ** 2 * weight)
    return loss_mse

def gen_random_color(min_value=0, max_value=256) -> str:
    """Generate random color for plotly"""
    r, g, b = np.random.randint(min_value, max_value, 3)
    return f'rgb({r},{g},{b})'

In [ ]:
def fit_sigmoid(exclude_days=0):
    target_country_df_list = []
    pred_df_list = []
    for target_country in top30_countries:
        print('target_country', target_country)
        # --- Train ---
        target_country_df = country_df.query('country == @target_country')

        #train_start_date = target_country_df['date'].min()
        train_start_date = target_country_df.query('confirmed > 1000')['date'].min()
        train_end_date = pd.to_datetime(target_date) - pd.Timedelta(f'{exclude_days} days')
        target_date_df = target_country_df.query('(date >= @train_start_date) & (date <= @train_end_date)')
        if len(target_date_df) <= 7:
            print('WARNING: the data is not enough, use 7 more days...')
            train_start_date -= pd.Timedelta('7 days')
            target_date_df = target_country_df.query('(date >= @train_start_date) & (date <= @train_end_date)')

        confirmed = target_date_df['confirmed'].values
        x = np.arange(len(confirmed))

        lossfun = lambda params: error(x, confirmed, params)
        res = sp.optimize.minimize(lossfun, x0=[np.max(confirmed) * 5, 0.04, 2 * len(confirmed) / 3.], method='nelder-mead')
        M, beta, alpha = res.x
        # sigmoid_models[key] = (M, beta, alpha)
        # np.clip(sigmoid(list(range(len(data), len(data) + steps)), M, beta, alpha), 0, None).astype(int)

        # --- Pred ---
        pred_start_date = target_country_df['date'].min()
        pred_end_date = pd.to_datetime('2021-06-01')
        days = int((pred_end_date - pred_start_date) / pd.Timedelta('1 days'))
        # print('pred start', pred_start_date, 'end', pred_end_date, 'days', days)

        x = np.arange(days)
        offset = (train_start_date - pred_start_date) / pd.Timedelta('1 days')
        print('train_start_date', train_start_date, 'offset', offset, 'params', M, beta, alpha)
        y_pred = sigmoid(x, M, beta, alpha, offset=offset)
        # target_country_df['confirmed_pred'] = y_pred

        all_dates = [pred_start_date + np.timedelta64(x, 'D') for x in range(days)]
        pred_df = pd.DataFrame({
            'date': all_dates,
            'country': target_country,
            'confirmed_pred': y_pred,
        })

        target_country_df_list.append(target_country_df)
        pred_df_list.append(pred_df)
    return target_country_df_list, pred_df_list

In [ ]:
def plot_sigmoid_fitting(target_country_df_list, pred_df_list, title=''):
    n_countries = len(top30_countries)

    # --- Plot ---
    fig = go.Figure()

    for i in range(n_countries):
        target_country = top30_countries[i]
        target_country_df = target_country_df_list[i]
        pred_df = pred_df_list[i]
        color = gen_random_color(min_value=20)
        # Prediction
        fig.add_trace(go.Scatter(
            x=pred_df['date'], y=pred_df['confirmed_pred'],
            name=f'{target_country}_pred',
            line=dict(color=color, dash='dash')
        ))

        # Ground truth
        fig.add_trace(go.Scatter(
            x=target_country_df['date'], y=target_country_df['confirmed'],
            mode='markers', name=f'{target_country}_actual',
            line=dict(color=color),
        ))
    fig.update_layout(
        title=title, xaxis_title='Date', yaxis_title='Confirmed cases')
    fig.show()

In [ ]:
target_country_df_list, pred_df_list = fit_sigmoid(exclude_days=0)

通过sigmoid函数我们能得到每个国家确诊数的预测值，我们的预测期数是144期，到2021-06-01。

In [ ]:
plot_sigmoid_fitting(target_country_df_list, pred_df_list, title='Sigmoid fitting with all latest data')

**图表分析：**绘制当前实际值与未来144期预测值的时间序列折线图如下，可以看到确诊病例正在放缓，而且全球大部分地区相同。就目前情势来看，如果美国和巴西没有根本性政策改变的话，确诊病例到6月初依旧会有增加。因此，要在全球范围内控制住疫情，还需要长期的努力。我们也面临着第二波挑战，也就是春节的人口大规模流动。

In [ ]:
target_country_df_list, pred_df_list = fit_sigmoid(exclude_days=7)

由于数据量太大，模型的检验部分我们才去直接采取带入实际值进行效果判定，具体方法是排除过去7天的数据来再次拟合模型进行判定。

In [ ]:
plot_sigmoid_fitting(target_country_df_list, pred_df_list, title='Sigmoid fitting without last 7days data')

**图表分析：**从图上我们可以注意到sigmoid拟合往往低估了曲线，确诊病例实际数值往往比sigmoid曲线估计值要大。因此，sigmoid曲线预测数据数据往往更加保守，实际情况很可能比之前用所有数据训练出的模型预测值更高，我们应该考虑更高的风险值。

## 3.2 短期预测

**说明：**对模型预测期数的讨论，长期期数下，预测效果一定存在很大的偏误，应该尽可能避免长期预测，而是采用短期预测，等待新样本加入模型之后再进行短期预测，才会有较高的预测精度。通过查找文献，我们尝试实现了几种方法来实现短期预测，有支持向量机预测，多项式函数预测，贝叶斯回归这三种方法，只用短期数据做了误差分析，得到的结论是支持向量机在处理短期确诊人数的预测时表现较为优良。

### 3.2.1 SVM预测

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('01-07-2021.csv')
us_medical_data = pd.read_csv('01-07-2021(2).csv')

cols = confirmed_df.keys()

confirmed = confirmed_df.loc[:, cols[4]:cols[-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]

dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)
    
    
def daily_increase(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d 

def moving_average(data, window_size):
    moving_average = []
    for i in range(len(data)):
        if i + window_size < len(data):
            moving_average.append(np.mean(data[i:i+window_size]))
        else:
            moving_average.append(np.mean(data[i:len(data)]))
    return moving_average

# window size
window = 7

# confirmed cases
world_daily_increase = daily_increase(world_cases)
world_confirmed_avg= moving_average(world_cases, window)
world_daily_increase_avg = moving_average(world_daily_increase, window)

# deaths
world_daily_death = daily_increase(total_deaths)
world_death_avg = moving_average(total_deaths, window)
world_daily_death_avg = moving_average(world_daily_death, window)


# recoveries
world_daily_recovery = daily_increase(total_recovered)
world_recovery_avg = moving_average(total_recovered, window)
world_daily_recovery_avg = moving_average(world_daily_recovery, window)


# active 
world_active_avg = moving_average(total_active, window)

days_since_1_22 = np.array([i for i in range(len(dates))]).reshape(-1, 1)
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)
total_recovered = np.array(total_recovered).reshape(-1, 1)

In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))
    
# slightly modify the data to fit the model better (regression models cannot pick the pattern)
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(days_since_1_22[50:], world_cases[50:], test_size=0.05, shuffle=False) 

In [ ]:
# svm_confirmed = svm_search.best_estimator_
svm_confirmed = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed.fit(X_train_confirmed, y_train_confirmed)
svm_pred = svm_confirmed.predict(future_forcast)

In [ ]:
# check against testing data
svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(y_test_confirmed)
plt.plot(svm_test_pred)
plt.legend(['Test Data', 'SVM Predictions'])
print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(svm_test_pred, y_test_confirmed))

### 3.2.1 Polynomial Regression

In [ ]:
# transform our data for polynomial regression
poly = PolynomialFeatures(degree=4)
poly_X_train_confirmed = poly.fit_transform(X_train_confirmed)
poly_X_test_confirmed = poly.fit_transform(X_test_confirmed)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly = PolynomialFeatures(degree=5)
bayesian_poly_X_train_confirmed = bayesian_poly.fit_transform(X_train_confirmed)
bayesian_poly_X_test_confirmed = bayesian_poly.fit_transform(X_test_confirmed)
bayesian_poly_future_forcast = bayesian_poly.fit_transform(future_forcast)

In [ ]:
# polynomial regression
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmed, y_train_confirmed)
test_linear_pred = linear_model.predict(poly_X_test_confirmed)
linear_pred = linear_model.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred, y_test_confirmed))

In [ ]:
print(linear_model.coef_)

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_linear_pred)
plt.legend(['Test Data', 'Polynomial Regression Predictions'])

### 3.2.3 Bayesian Ridge Polynomial Regression

In [ ]:
# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian = BayesianRidge(fit_intercept=False)
bayesian_search = RandomizedSearchCV(bayesian, bayesian_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search.fit(bayesian_poly_X_train_confirmed, y_train_confirmed)

In [ ]:
bayesian_search.best_params_

In [ ]:
bayesian_confirmed = bayesian_search.best_estimator_
test_bayesian_pred = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed)
bayesian_pred = bayesian_confirmed.predict(bayesian_poly_future_forcast)
print('MAE:', mean_absolute_error(test_bayesian_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_bayesian_pred, y_test_confirmed))

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_bayesian_pred)
plt.legend(['Test Data', 'Bayesian Ridge Polynomial Predictions'])

用短期数据做了误差分析，得到的结论是支持向量机在处理短期确诊人数的预测时表现较为优良。

**章节小结：**  
1.对于确诊人数的预测我们还存在很多可以改进的地方。比如对模型的稳健性和灵敏度的讨论，考虑不同国家未来抗疫政策对模型的影响，考虑例如春节这种会对疫情带来显著影响的强影响事件。  
2.通过查找文献，我们还了解到其他类别的方法，比如利用社会学中信息传播模型来类比建立关于新冠肺炎的传播模型，因为每个感染者/创新采纳者的接触人数似乎是相关的。

<br>

<a id="id_medicine"></a>
# 4. 生物统计行业

## 4.1 生物统计相关技能要求

In [ ]:
#代码部分需要记得更改文件路径

In [ ]:
import os
from os import path
import numpy as np
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from PIL import Image
from matplotlib import pyplot as plt
from imageio import imread
import random

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    # 获取文本text
#text = open(path.join(d,'JD信息.txt')).read()
text=open('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/JD信息.txt').read()
    # 读取背景图片
background_Image = imread('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/可视化.png')
    # 提取背景图片颜色
img_colors = ImageColorGenerator(background_Image)
    # 设置英文停止词
stopwords = set(STOPWORDS)
stopwords.add('ability')
stopwords.add('able')
stopwords.add('interests')
stopwords.add('address')
stopwords.add('carries')
stopwords.add('explanation')
stopwords.add('generalization')
stopwords.add('preparation')
stopwords.add('abstracts')
stopwords.add('duties')
stopwords.add('work')
#print(stopwords)


wc = WordCloud(
    width=1000,
    height=700,
    margin = 2, # 设置页面边缘
    mask = background_Image,
    scale = 30,
    max_words = 20, # 最多词个数
    min_font_size = 4, # 最小字体大小
    stopwords = stopwords,
    random_state = 42,
    background_color = 'white', # 背景颜色
    max_font_size = 150, # 最大字体大小
    colormap = 'RdYlGn'
    )
    # 生成词云
wc.generate_from_text(text)
    # 等价于
    # wc.generate(text)
    # 根据图片色设置背景色
#wc.recolor(color_func=img_colors)
    #存储图像
wc.to_file('JD信息词云图.png')
    # 显示图像

plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

除了统计软件之外，出版物、设备操作、热情也是求职生物统计岗位的重要因素。

## 4.2 生物统计相关岗位

In [ ]:
import os
from os import path
import numpy as np
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from PIL import Image
from matplotlib import pyplot as plt
from imageio import imread
import random

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    # 获取文本text
#text = open(path.join(d,'岗位名称.txt')).read()
text=open('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/岗位名称.txt').read()
    # 读取背景图片
background_Image = imread('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/可视化.png')
    # 提取背景图片颜色
img_colors = ImageColorGenerator(background_Image)
    # 设置英文停止词
stopwords = set(STOPWORDS)
#print(stopwords)


wc = WordCloud(
    width=1000,
    height=700,
    margin = 2, # 设置页面边缘
    mask = background_Image,
    scale = 30,
    max_words = 20, # 最多词个数
    min_font_size = 4, # 最小字体大小
    stopwords = stopwords,
    random_state = 42,
    background_color = 'white', # 背景颜色
    max_font_size = 150, # 最大字体大小
    colormap = 'winter'
    )
    # 生成词云
wc.generate_from_text(text)
    # 等价于
    # wc.generate(text)
    # 根据图片色设置背景色
#wc.recolor(color_func=img_colors)
    #存储图像
wc.to_file('岗位名称词云图.png')
    # 显示图像

plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

生物统计相关工作不只局限在生物统计，数据科学家、生物信息学家都有一席之地。

## 4.3 生物统计相关公司

In [ ]:
import os
from os import path
import numpy as np
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from PIL import Image
from matplotlib import pyplot as plt
from imageio import imread
import random

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    # 获取文本text
#text = open(path.join(d,'公司名称.txt')).read()
text=open('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/公司名称.txt').read()
    # 读取背景图片
background_Image = imread('/Users/haoranzhang/人大ISBD/数据可视化/数据可视化final project+董瑶_刘昀霖_张浩然/Data/可视化.png')
    # 提取背景图片颜色
img_colors = ImageColorGenerator(background_Image)
    # 设置英文停止词
stopwords = set(STOPWORDS)
#print(stopwords)


wc = WordCloud(
    width=1000,
    height=700,
    margin = 2, # 设置页面边缘
    mask = background_Image,
    scale = 30,
    max_words = 20, # 最多词个数
    min_font_size = 4, # 最小字体大小
    stopwords = stopwords,
    random_state = 42,
    background_color = 'white', # 背景颜色
    max_font_size = 150, # 最大字体大小
    colormap = 'viridis'
    )
    # 生成词云
wc.generate_from_text(text)
    # 等价于
    # wc.generate(text)
    # 根据图片色设置背景色
#wc.recolor(color_func=img_colors)
    #存储图像
wc.to_file('公司名称词云图.png')
    # 显示图像

plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()

大型外企药厂是招收生物统计职位最多的公司，例如拜耳等。高校实验室和医院也有一定的需求。